# Note
This processing was done without setting any time offset between Otter and video data, and this probably led to incorrect georeferencing of extracted images. However, this notebook is kept unchanged as a reference.

In [1]:
import video_transect, misc
from pathlib import Path
import pandas as pd

In [2]:
# Paths
transect_dirs = [Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/OtterTransects/20230619/VideoTransects/Transect1'),
                 Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/OtterTransects/20230619/VideoTransects/Transect2')]
csv_dir = Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/OtterTransects/20230619/GnssData')


In [3]:
# Parameters
sample_distance = 1.0 # Units defined by CRS, default is UTM with unit meters

In [4]:
# Read all CSV files with Otter positions, and merge (concatenate) them
csv_files = sorted(csv_dir.glob('*.csv'))
gdfs = []
for csv_file in csv_files:
    gdfs.append(video_transect.otter_csv_to_geodataframe(csv_file))
otter_gdf = pd.concat(gdfs,axis=0,ignore_index=True)
otter_gdf = video_transect.filter_gdf_on_distance(otter_gdf,sample_distance)
otter_gdf

,Time,Lat,Lng,geometry
0,2023-06-19 12:24:08.756000+00:00,63.408393,7.730441,POINT (7.73044 63.40839)
8,2023-06-19 12:24:09.538000+00:00,63.408391,7.730462,POINT (7.73046 63.40839)
16,2023-06-19 12:24:10.338000+00:00,63.408388,7.730483,POINT (7.73048 63.40839)
24,2023-06-19 12:24:11.138000+00:00,63.408385,7.730504,POINT (7.73050 63.40839)
32,2023-06-19 12:24:11.938000+00:00,63.408382,7.730524,POINT (7.73052 63.40838)
...,...,...,...,...
226783,2023-06-19 19:15:22.194000+00:00,63.407716,7.737913,POINT (7.73791 63.40772)
226886,2023-06-19 19:15:32.497000+00:00,63.407708,7.737919,POINT (7.73792 63.40771)
226975,2023-06-19 19:15:41.392000+00:00,63.407715,7.737930,POINT (7.73793 63.40772)
226993,2023-06-19 19:15:43.200000+00:00,63.407722,7.737943,POINT (7.73794 63.40772)


In [5]:
# NOTE: Do a "dry run" without extracting images and check that time offsets are correct

for transect_dir in transect_dirs:
    video_dir = transect_dir / 'Video'
    
    image_dir = transect_dir / 'ExtractedImages/Original'
    image_dir.mkdir(parents=True, exist_ok=True)
    print(f'{image_dir} created (if not created before)')

    gpkg_dir = transect_dir / 'ImagesGeopackage'
    gpkg_dir.mkdir(exist_ok=True)
    print(f'{gpkg_dir} created (if not created before)')

    gpkg_file = gpkg_dir / (transect_dir.stem + '_GeotaggedImages.gpkg')
    print(f'GPKG file name for saving results: {gpkg_file}')

    video_files = misc.file_pattern_search(str(video_dir), '*.[Mm][Pp]4')
    print(f'Found video files {[Path(vfile).name for vfile in video_files]}')
    video_data = video_transect.get_video_data(str(video_dir),tz='UTC')
    
    gdf = video_transect.prepare_gdf_with_video_data(otter_gdf,video_data)
    gdf = video_transect.extract_images_from_video(gdf,str(image_dir))
    gdf.to_file(gpkg_file, driver="GPKG")
    

/media/mha114/Massimal2/Massimal/Smola_Maholmen/OtterTransects/20230619/VideoTransects/Transect1/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Maholmen/OtterTransects/20230619/VideoTransects/Transect1/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Maholmen/OtterTransects/20230619/VideoTransects/Transect1/ImagesGeopackage/Transect1_GeotaggedImages.gpkg
Found video files ['GX015397.MP4', 'GX025397.MP4', 'GX035397.MP4', 'GX045397.MP4', 'GX055397.MP4', 'GX065397.MP4']


100%|██████████| 1739/1739 [22:15<00:00,  1.30it/s]


/media/mha114/Massimal2/Massimal/Smola_Maholmen/OtterTransects/20230619/VideoTransects/Transect2/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Maholmen/OtterTransects/20230619/VideoTransects/Transect2/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Maholmen/OtterTransects/20230619/VideoTransects/Transect2/ImagesGeopackage/Transect2_GeotaggedImages.gpkg
Found video files ['GX010028.MP4', 'GX020028.MP4', 'GX030028.MP4', 'GX040028.MP4', 'GX050028.MP4']


100%|██████████| 3314/3314 [41:54<00:00,  1.32it/s]  
